In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/combineddataset/CombinedDataset.csv
/kaggle/input/final-dataset/FinalDataset.csv


In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [58]:
df = pd.read_csv('/kaggle/input/final-dataset/FinalDataset.csv')

In [59]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Extract input (features) and output (real & imaginary parts)
X = df[['w_Si(nm)', 'h_Si(nm)']].values
y = df[['Refractive Index(real part)', 'Refractive Index(imaginary part) ']].values

# Normalize input features
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Define Transformer Block
def transformer_block(x, head_size=64, num_heads=16, ff_dim=128):
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    #x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x
    x = layers.Dense(ff_dim, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.Dense(head_size, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001))(x)
    #x = layers.Dropout(dropout)(x)
    x = layers.Add()([x, res])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x

# Define Squeeze-and-Excitation (SE) Block
def se_block(x, ratio=16):
    filters = x.shape[-1]
    se = layers.GlobalAveragePooling1D()(x)
    se = layers.Dense(filters // ratio, activation="swish")(se)
    se = layers.Dense(filters, activation="swish")(se)
    return layers.Multiply()([x, se])

# Define CAT-CNN Model
def build_model():
    input_layer = layers.Input(shape=(2, 1))
    
    # CNN Feature Extraction
    x = layers.Conv1D(filters=64, kernel_size=2, activation='swish', padding='same')(input_layer)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='swish', padding='same')(x)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='swish', padding='same')(x)
    x = layers.Conv1D(filters=64, kernel_size=2, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    # Apply SE Block
    x = se_block(x)
    
    # Transformer Blocks
    x = transformer_block(x)
    x = transformer_block(x)

    x = layers.GlobalAveragePooling1D()(x)
    
    # Fully Connected Layers
    x = layers.Dense(128, activation='swish', kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.Dropout(0.2)(x)
    
    # Output Layers (Real & Imaginary Parts)
    real_part_output = layers.Dense(1, name='real_part')(x)
    imag_part_output = layers.Dense(1, name='imag_part')(x)
    
    # Define Model
    model = keras.Model(inputs=input_layer, outputs=[real_part_output, imag_part_output])
    
    # Cosine Decay Learning Rate
    lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate=0.0001, first_decay_steps=50, t_mul=2.0, m_mul=0.9
    )
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    
    # Compile Model
    model.compile(optimizer=optimizer,
                  loss=[keras.losses.MeanSquaredError(), keras.losses.Huber()],
                  loss_weights=[2.0, 0.5],
                  metrics=[['mae'], ['mae']])
    
    return model

# Build and Train Model
model = build_model()
history = model.fit(X_train.reshape(-1, 2, 1), [y_train[:, 0], y_train[:, 1]], 
                    validation_data=(X_test.reshape(-1, 2, 1), [y_test[:, 0], y_test[:, 1]]), 
                    epochs=500, batch_size=32)

# Evaluate the model
y_pred_real, y_pred_imag = model.predict(X_test.reshape(-1, 2, 1))
y_pred = np.column_stack([y_pred_real, y_pred_imag])
y_pred = scaler_y.inverse_transform(y_pred)
y_test_original = scaler_y.inverse_transform(y_test)

# Compute R² scores
r2_real = r2_score(y_test_original[:, 0], y_pred[:, 0])
r2_imag = r2_score(y_test_original[:, 1], y_pred[:, 1])
var_real = np.var(y_test_original[:, 0])
var_imag = np.var(y_test_original[:, 1])
r2_combined = (var_real * r2_real + var_imag * r2_imag) / (var_real + var_imag)

print(f"Testing R² (Real Part): {r2_real}")
print(f"Testing R² (Imaginary Part): {r2_imag}")
print(f"Combined Testing R² Score: {r2_combined}")

Epoch 1/500
186/186 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - imag_part_loss: 0.1514 - imag_part_mae: 0.6302 - loss: 2.0729 - real_part_loss: 1.4109 - real_part_mae: 0.6471 - val_imag_part_loss: 0.2190 - val_imag_part_mae: 0.7965 - val_loss: 2.8986 - val_real_part_loss: 2.1828 - val_real_part_mae: 0.8395
Epoch 2/500
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - imag_part_loss: 0.0306 - imag_part_mae: 0.2528 - loss: 0.7286 - real_part_loss: 0.1922 - real_part_mae: 0.2381 - val_imag_part_loss: 0.1786 - val_imag_part_mae: 0.7206 - val_loss: 2.2773 - val_real_part_loss: 1.6021 - val_real_part_mae: 0.7585
Epoch 3/500
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - imag_part_loss: 0.0261 - imag_part_mae: 0.2325 - loss: 0.6751 - real_part_loss: 0.1479 - real_part_mae: 0.2063 - val_imag_part_loss: 0.0953 - val_imag_part_mae: 0.4636 - val_loss: 1.3547 - val_real_part_loss: 0.7672 - val_real_part_mae: 0.4850
Epoch 4/500
186/186 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - imag_part_loss: 0.0223 - imag_part_mae: 0.2156

In [60]:
# Predict refractive index for given input
input_sample = np.array([[205,220]])
#input_expanded = np.concatenate([input_sample, input_sample**2, np.sqrt(input_sample)], axis=1)
input_scaled = scaler_X.transform(input_sample)
pred_real, pred_imag = model.predict(input_scaled.reshape(-1, 2, 1))
pred_inverse = scaler_y.inverse_transform(np.column_stack([pred_real, pred_imag]))

print(f"Predicted Refractive Index - Real: {pred_inverse[0, 0]}, Imaginary: {pred_inverse[0, 1]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
Predicted Refractive Index - Real: 2.217494010925293, Imaginary: -0.04367320239543915
